In [ ]:
import log_message_pb2

### Odrive Setup

In [35]:
import odrive.enums
import numpy as np
from enum import Enum


class ErrorType(Enum):
    AXIS = 0
    MOTOR = 1
    SENSORLESS_ESTIMATOR = 2
    ENCODER = 3
    CONTROLLER = 4


error_type_infos = {
    ErrorType.MOTOR: ("motor_error", "Motor Errors"),
    ErrorType.AXIS: ("axis_error", "Axis Errors"),
    ErrorType.ENCODER: ("encoder_error", "Encoder Errors"),
}


def printErrors(error, error_type, prefix=""):
    error = int(error)
    if error_type == ErrorType.AXIS:
        error_map_prefix = "AXIS_ERROR_"
    elif error_type == ErrorType.MOTOR:
        error_map_prefix = "MOTOR_ERROR_"
    elif error_type == ErrorType.SENSORLESS_ESTIMATOR:
        error_map_prefix = "SENSORLESS_ESTIMATOR_ERROR_"
    elif error_type == ErrorType.ENCODER:
        error_map_prefix = "ENCODER_ERROR_"
    elif error_type == ErrorType.CONTROLLER:
        error_map_prefix = "CONTROLLER_ERROR_"
    error_map = {
        v: k for k, v in odrive.enums.__dict__.items() if k.startswith(error_map_prefix)
    }
    for bit in range(64):
        if error & (1 << bit) != 0:
            print(f"{prefix}{error_map.get(1 << bit)}")


def printErrorTimes(df, error_type):
    error_type_info = error_type_infos[error_type]
    idxs = np.where(df[error_type_info[0]] != df[error_type_info[0]].shift())

    for idx in idxs[0]:
        error_time_s = df.iloc[idx]["start_s"]
        error = df.iloc[idx][error_type_info[0]]
        print(f"{error_type_info[1]} ({error_time_s:.02f}):")
        if error == 0:
            print("  NONE")
        else:
            printErrors(error, error_type, "  ")

### Data Parsing Setup

In [6]:
import log_message_pb2

['control_cycle_count', 'control_cycle_start_us', 'control_cycle_stop_us', 'control_cycle_dt_us', 'wheel_rpm', 'engine_rpm', 'engine_count', 'wheel_count', 'target_rpm', 'velocity_command', 'last_heartbeat_ms', 'axis_error', 'motor_error', 'encoder_error', 'voltage', 'iq_measured', 'iq_setpoint', 'odrive_current', 'inbound_estop', 'outbound_estop', 'shadow_count']


In [1]:
import os
import pandas as pd

import plotly.graph_objects as go
import plotly.io as pio

import log_message_pb2
import header_message_pb2

pio.templates.default = "plotly_dark"


# import plotly.express as px
# import plotly.subplots as subplots

csv_cols = [
    "dt_us",
    "voltage",
    "heartbeat",
    "wheel_rpm",
    "engine_rpm",
    "target_rpm",
    "velocity_command",
    "real_velocity_command",
    "shadow_count",
    "ignore1",
    "ignore2",
    "iq_measured",
    "flushed",
    "wheel_count",
    "engine_count",
    "iq_setpoint",
    "start_us",
    "stop_us",
    "current",
    "axis_error",
    "motor_error",
    "encoder_error",
]

wheel_diameter = 23
secondary_ratio = 12 / 6 * 45 / 17
wheel_ratio = 12 / 6 * 18 / 57
pitch_angle = 5
encoder_cpr = 8192

In [37]:
class DescribedDataframe(pd.DataFrame):
    _metadata = ["filename", "description"]

    @property
    def _constructor(self):
        return DescribedDataframe

In [38]:
def parseCSVFile(path):
    try:
        df = pd.read_csv(path, skiprows=1, header=None, names=csv_cols)
    except:
        return None
    return DescribedDataframe(df)


def parseBinaryFile(path):
    with open(path, "rb") as file:
        while True:
            message_type_raw = file.read(1)
            if message_type_raw == b"":
                break
            message_length_raw = file.read(4)

            message_type = int(message_type_raw, 16)
            message_length = int(message_length_raw, 16)
            message = file.read(message_length)

            if message_type == 0:
                header_message = header_message_pb2.HeaderMessage()
                header_message.ParseFromString(message)
                print(
                    f"{header_message.timestamp_human}, {header_message.clock_us}, {header_message.p_gain}"
                )
            elif message_type == 1:
                message_type = log_message_pb2.LogMessage.DESCRIPTOR
                log_message = log_message_pb2.LogMessage()
                log_message.ParseFromString(message)
                
                """
                for field in message_type.fields:
                    field_name = field.name
                    field_value = getattr(log_message, field_name)
                    print(f"{field_name}: {field_value}")
                """
                print(f"{log_message.control_cycle_count},", end="")
                print(f"{log_message.control_cycle_start_us},", end="")
                print(f"{log_message.control_cycle_stop_us},", end="")
                print(f"{log_message.control_cycle_dt_us},", end="")
                print(f"{log_message.control_cycle_dt_us},", end="")
                print(f"{log_message.wheel_rpm},", end="")
                print(f"{log_message.engine_rpm},", end="")
                print(f"{log_message.engine_count},", end="")
                print(f"{log_message.wheel_count},", end="")
                print(f"{log_message.target_rpm},", end="")
                print(f"{log_message.velocity_command},", end="")
                print(f"{log_message.last_heartbeat_ms},", end="")
                print(f"{log_message.axis_error},", end="")
                print(f"{log_message.motor_error},", end="")
                print(f"{log_message.encoder_error},", end="")
                print(f"{log_message.voltage},", end="")
                print(f"{log_message.iq_measured},", end="")
                print(f"{log_message.iq_setpoint},", end="")
                print(f"{log_message.odrive_current},", end="")
                print(f"{log_message.inbound_estop},", end="")
                print(f"{log_message.outbound_estop},", end="")
                print(f"{log_message.shadow_count},", end="")
                print("")


def postProcessDataframe(df):
    df["start_s"] = df["start_us"] / 1e6

    df["secondary_rpm"] = df["wheel_rpm"] * secondary_ratio
    df["wheel_rpm"] = df["wheel_rpm"] * wheel_ratio
    df["wheel_mph"] = (df["wheel_rpm"] * wheel_diameter * np.pi) / (12 * 5280) * 60

    df["actuator_position_mm"] = -df["shadow_count"] / encoder_cpr * pitch_angle
    df["shift_ratio"] = df["secondary_rpm"] / df["engine_rpm"]

    motor_error_idxs = np.where(df["motor_error"] != 0)
    motor_error_bool = np.zeros(len(df))
    motor_error_bool[motor_error_idxs] = 1
    df["motor_error_bool"] = motor_error_bool


def addNormalizedColumns(df):
    for col in df:
        col_norm = f"norm_{col}"
        if col.startswith("norm_") or col_norm in df:
            continue
        col_obj = df[col]
        df[col_norm] = (col_obj - np.min(col_obj)) / (np.max(col_obj) - np.min(col_obj))

In [39]:
parseBinaryFile("logs/file.bin")

2023-04-18 15:36:38, 4595621, 0.01600000075995922
0,9696241,9696244,100002,100002,0.0,0.0,0,0,3500.0,0.0,9696,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
1,9796241,9796242,100000,100000,0.0,0.0,0,0,3500.0,0.0,9796,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
2,9896241,9896242,100000,100000,0.0,0.0,0,0,3500.0,0.0,9896,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
3,9996241,9996242,100000,100000,0.0,0.0,0,0,3500.0,0.0,9996,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
4,10096241,10096242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10096,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
5,10196241,10196242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10196,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
6,10296241,10296242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10296,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
7,10396241,10396242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10396,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
8,10496241,10496242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10496,0,0,0,0.0,0.0,0.0,0.0,False,False,0,
9,10596241,10596242,100000,100000,0.0,0.0,0,0,3500.0,0.0,10596

### Parse Data

In [39]:
def getTextLogs(log_dir):
    log_dir_contents = os.listdir(log_dir)
    txt_paths = []
    for potential_file in log_dir_contents:
        potential_file_path = os.path.join(log_dir, potential_file)
        potential_file_ext = os.path.splitext(potential_file)[1]
        if os.path.isfile(potential_file_path) and potential_file_ext == ".txt":
            txt_paths.append(potential_file_path)
    txt_paths.sort()
    return txt_paths


def filterFilesBySize(paths, size_kb):
    return [path for path in paths if os.path.getsize(path) >= size_kb * 1e3]

In [40]:
log_dir = "logs/"
# log_dir = "/home/grant/tmp"
generate_html = True
offline = False
show_figures = False
min_log_size_kb = 10

paths = getTextLogs(log_dir)
paths = filterFilesBySize(paths, min_log_size_kb)

In [41]:
dfs = [] 
# hehe troll

for path in paths:
    df = parseCSVFile(path)
    if df is None:
        continue
    postProcessDataframe(df)
    df.filename = os.path.basename(path)
    addNormalizedColumns(df)
    dfs.append(df)

### Figure Generators

In [42]:
def getRPMFigure(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["engine_rpm"], name="Engine RPM"))
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(go.Scatter(x=df["start_s"], y=df["target_rpm"], name="Target RPM"))
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="RPM",
        title=f"Engine RPM and Secondary RPM<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [43]:
def getRPMAndActuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_engine_rpm"], name="Engine RPM")
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_secondary_rpm"], name="Secondary RPM")
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["norm_actuator_position_mm"], name="Actuator Position"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Engine RPM, Secondary RPM, and Actuator Position<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [44]:
def getVehicleSpeedFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["wheel_mph"],
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Vehicle Speed (mph)",
        title=f"Vehicle Speed<br><sup>{df.filename}</sup>",
    )
    return fig

In [45]:
def getShiftRatioAndAcuatorFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"],
            y=df["norm_actuator_position_mm"],
            name="Actuator Position (mm)",
        )
    )
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["norm_shift_ratio"], name="Shift Ratio")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        title=f"Normalized Actuator Position and Shift Ratio<br><sup>{df.filename}</sup>",
    )
    fig.update_traces(showlegend=True)
    return fig

In [46]:
def getVelocityCommandsFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["velocity_command"], name="Unclamped Velocity Command"
        )
    )
    fig.add_trace(
        go.Scatter(
            x=df["start_s"], y=df["real_velocity_command"], name="Velocity Command"
        )
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Velocity Command",
        title=f"Velocity Command<br><sup>{df.filename}</sup>",
    )
    return fig

In [47]:
def getShadowCountFigure(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=df["start_s"], y=df["shadow_count"], name="Shadow Count")
    )
    fig.update_layout(
        xaxis_title="Time (s)",
        yaxis_title="Shadow Count",
        title=f"Shadow Count<br><sup>{df.filename}</sup>",
    )
    return fig

In [48]:
def getEngineVsWheel(df):
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=df["wheel_mph"], y=df["engine_rpm"], name="Engine RPM vs Wheel Speed"
        )
    )
    fig.update_layout(
        xaxis_title="Wheel Speed (mph)",
        yaxis_title="Engine RPM",
        title=f"Engine RPM vs Wheel Speed<br><sup>{df.filename}</sup>",
    )
    return fig


### Create Invidual Graphs (By Logs)

In [49]:
def figuresToHTML(figs, filename, offline=False):
    with open(filename, "w") as file:
        file.write("<html><head></head><body>" + "\n")
        for fig in figs:
            font_size_backup = fig.layout.font.size
            fig.update_layout(font={"size": 20})
            inner_html = (
                fig.to_html(include_plotlyjs=(True if offline else "cdn"))
                .split("<body>")[1]
                .split("</body>")[0]
            )
            file.write(inner_html)
            fig.update_layout(font={"size": font_size_backup})
        file.write("</body></html>" + "\n")

In [50]:
all_figs_by_log = []
for df in dfs:
    all_figs_by_log.append(
        [
            getRPMFigure(df),
            getRPMAndActuatorFigure(df),
            getVehicleSpeedFigure(df),
            getShiftRatioAndAcuatorFigure(df),
            getVelocityCommandsFigure(df),
            getShadowCountFigure(df),
            getEngineVsWheel(df),
        ]
    )

In [51]:
if generate_html:
    for path, figs_by_log in zip(paths, all_figs_by_log):
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{filename_without_ext}.html"
        figuresToHTML(figs_by_log, html_path, offline=offline)

In [63]:
b1 = widgets.Button(description='Custom color')
b1.style.button_color = 'lightgreen'
b1



Button(description='Custom color', style=ButtonStyle(button_color='lightgreen'))

In [70]:

%%html
<style>
    .widget-select > select {background-color: red;}   
</style>

In [74]:
import ipywidgets as widgets
from IPython.display import clear_output

show_figures = True
if show_figures:
    dropdown = widgets.Dropdown(options=paths, value=paths[-1])


    def onDropdownChange(change):
        if change["type"] == "change" and change["name"] == "value":
            clear_output()
            idx = paths.index(change["new"])
            printErrorTimes(dfs[idx], ErrorType.MOTOR)
            display(dropdown)
            for fig in all_figs_by_log[idx]:
                fig.show()

    idx = -1
    dropdown.observe(onDropdownChange)
    printErrorTimes(dfs[idx], ErrorType.MOTOR)
    display(dropdown)
    for fig in all_figs_by_log[idx]:
        fig.show()

Motor Errors (3.62):
  NONE


Dropdown(index=26, options=('logs/log_2019-01-01_00-05-25.txt', 'logs/log_2023-04-13_13-22-30.txt', 'logs/log_…

### Create Group Graphs (By Graph Type)

In [53]:
figure_names_and_funcs = [
    ("rpm", getRPMFigure),
    ("rpm-and-actuator", getRPMAndActuatorFigure),
    ("vehicle-speed", getVehicleSpeedFigure),
    ("shift-ratio-and-acuator", getShiftRatioAndAcuatorFigure),
    ("velocity-command", getVelocityCommandsFigure),
    ("shadow-count", getShadowCountFigure),
    ("engine-vs-wheel", getEngineVsWheel),
]
all_figs_by_type = []
for name, func in figure_names_and_funcs:
    figs_by_type = []
    for df in dfs:
        figs_by_type.append(func(df))
    all_figs_by_type.append((name, figs_by_type))

In [54]:
if generate_html:
    for figs_by_type in all_figs_by_type:
        filename_without_ext = os.path.splitext(os.path.basename(path))[0]
        html_path = f"graphs/{figs_by_type[0]}.html"
        figuresToHTML(figs_by_type[1], html_path, offline=offline)

### Odrive Helpers

### Error Logging

In [55]:
df = dfs[-2]
printErrorTimes(df, ErrorType.MOTOR)

Motor Errors (5.57):
  NONE
Motor Errors (103.77):
  MOTOR_ERROR_UNKNOWN_TORQUE
